In [1]:
import random as rd
import numpy as np

from glob import glob

In [2]:
def euclides_extendido(a, b):
    '''
    Realiza o cálculo do mdc
    entre a e b, retornando
    o mdc e x e y tais que
    ax + by = mdc(a, b).
    '''
    invertido = False
    if b > a:
        a, b = b, a
        invertido = True
        
    
    tabela = np.array([[a, b], [1, 0], [0, 1]])
    iteracao = 0
    while tabela[0, (iteracao + 1) % 2] != 0:
        a, b = tabela[0, iteracao % 2], tabela[0, (iteracao + 1) % 2]
        q = a // b
        r = a % b
        tabela[:, iteracao % 2] -= tabela[:, (iteracao + 1) % 2] * q
        iteracao += 1
        
    mdc, x, y = tabela[:, iteracao % 2]
    
    if invertido:
        return mdc, y, x
    else:
        return mdc, x, y

In [3]:
def e_primo(n):
    '''
    Recebe um número e verifica
    se ele é primo
    '''
    if n == 2:
        return True
    
    if n < 2 or n % 2 == 0:
        return False
    
    for i in range(3, int(n**(0.5)) + 1, 2):
        if n % i == 0:
            return False
        
    return True

In [4]:
# Buscando os primos de 10.000 até
# 19.999

if 'primos.npy' not in glob('*.npy'):
    primos = []
    a = 10000
    for i in range(5000):
        if e_primo(a + 2 * i + 1):
            primos.append(a + 2 * i + 1)

    primos = np.array(primos)
    with open('primos.npy', 'wb') as f:
        np.save(f, primos)

else:
    primos = np.load('primos.npy')

primos

array([10007, 10009, 10037, ..., 19991, 19993, 19997])

In [5]:
def escolhe_parametros(primos):
    '''
    Recebe um array de primos e
    retorna dois primos p e q,
    phi(n) e números e e d de
    modo que de = 1 (mod phi(n)),
    onde n = p*q
    '''
    
    rd.shuffle(primos)
    p, q = primos[:2]
    phi_n = (p - 1) * (q - 1)
    
    e = 2
    mdc, _, d = euclides_extendido(phi_n, e)
    while mdc != 1:
        e = rd.randint(3, phi_n)
        mdc, _, d = euclides_extendido(phi_n, e)
        
    return p, q, phi_n, e, d, mdc, _

In [6]:
p, q, phi_n, e, d, mdc, _ = escolhe_parametros(primos)

mdc

1

In [7]:
(phi_n * _ + e * d) % phi_n

1

In [8]:
(phi_n * _) % phi_n

0

In [9]:
(e * d) % phi_n

1